In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [2]:
with open('shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(f"Number of characters is: {len(text)}")

Number of characters is: 1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
for i in range(30, 70):
    print(i,text[i])

30 e
31 d
32  
33 a
34 n
35 y
36  
37 f
38 u
39 r
40 t
41 h
42 e
43 r
44 ,
45  
46 h
47 e
48 a
49 r
50  
51 m
52 e
53  
54 s
55 p
56 e
57 a
58 k
59 .
60 

61 

62 A
63 l
64 l
65 :
66 

67 S
68 p
69 e


In [6]:
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
chars

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [8]:
s_to_i = {s:i for i,s in enumerate(chars)}
i_to_s = {i:s for i,s in enumerate(chars)}
encode = lambda s: [s_to_i[c] for c in s]
decode = lambda l: ''.join([i_to_s[i] for i in l])

print(encode("hello there"))
print(decode(encode("hii there")))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
hii there


In [9]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [10]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target is {target}.")

When input is tensor([18]) the target is 47.
When input is tensor([18, 47]) the target is 56.
When input is tensor([18, 47, 56]) the target is 57.
When input is tensor([18, 47, 56, 57]) the target is 58.
When input is tensor([18, 47, 56, 57, 58]) the target is 1.
When input is tensor([18, 47, 56, 57, 58,  1]) the target is 15.
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47.
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58.


In [12]:
# torch.manual_seed(1337)
# batch_size = 4
# block_size = 8

# def get_batch(split):
#     data = train_data if split == 'train' else val_data
#     ix = torch.randint(len(data) - block_size, (batch_size,)) #[num, num, ... 8 times]
#     x = torch.stack([data[i:i+block_size] for i in ix])
#     y = torch.stack([data[i+1:i+block_size+1] for i in ix])
#     return x,y

# xb, yb = get_batch('train')
# print(f'Inputs: {xb} \nInput Shape: {xb.shape}')
# print(f"Targets: {yb} \nTarget Shape: {yb.shape}")
# print('-------')

# for b in range(batch_size): #batch dimension
#     for t in range(block_size): #time dimension
#         context = xb[b, :t+1]
#         target = yb[b,t]
#         print(f"When input is {context.tolist()} the target is {target}")

In [13]:
#hyperparameters
batch_size = 64       #no. of independent seq we process in parallel
block_size = 256      #max context length for pred
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [14]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) #[num, num, ... 8 times]
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y =  x.to(device), y.to(device)
    return x,y

@torch.no_grad() # don't call .backward when doing estimate_loss()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out    

In [15]:
class Head(nn.Module):
    #one head of self-attention

    def __init__(self, head_size):
        super().__init__() #nn.Module's __init__
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) #lower triangular matrix of ones
        self.dropout = nn.Dropout(dropout) #think of dropout as a regularization/ optimization technique (kind of like random forest)

    def forward(self, x): #in karpathy's tutorials he assumes C and head_size are same but C is actually embedding dimension
        B,T,C = x.shape
        k = self.key(x) #(B,T,head_size)
        q = self.query(x) #(B,T,head_size)
        wei = q @ k.transpose(-2,-1) * (k.shape[-1] **-0.5) #(B,T,head_size) @ (B,head_size,T) --> (B,T,T) *C**-0.5 is done to ensure that before softmax the variance is 1
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) #(B,T,T) -inf so that softmax outputs 0 for them
        wei = F.softmax(wei, dim=-1) #(B,T,T)
        wei = self.dropout(wei)
        v = self.value(x) #(B,T,head_size)
        out = wei @ v #(B,T,head_size)
        #so out[0,2,7] means for the first batch the 3rd letter 8th dimension is the weighted sum of 8th dimesion of the value vectors for letters 0,1,2 
        #where the weights are computed using the query of token 2 and keys of tokens 0-2
        return out

Multi-head attention lets the model attend to different types of relationships simultaneously, which a single large attention head fundamentally cannot do (even though the number of dimensions match) because single head produces only one attention atrix and just one attention matrix cannout represent multiple independent attention patterns.

All the dimensions (say 32) of each letter get the same attention (or weights) in self attention but by dividng it into multiple heads of 8 dims each we can give these 4 batches of 8 dims different sets of attentions (or weights).

Assume that first head is for syntax and second head is for semantics. Say letter 3 must attend strongly (say 0.75) for syntax and weakly (say 0.03) for semantics to letter 1 and strongly (say 0.70) to letter 1 for semantics and weakly (say 0.05) for syntax to letter 2. This won't be possible in single head of 32 because the entire letter (or all 32 dimensions) will get attented to by the same weight and the sum of weights of syntax attention and semantics attention would be greater than 1 which is not possible because of softmaxing all 32 dims.

In [16]:
class MultiHeadAttention(nn.Module):
    #multiple heads of self-attention in parallel

    def __init__(self, num_heads, head_size):
        super().__init__() #nn.Module's __init__
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd) #this is done so that the concatenated head outputs back into the model's working 
        #embedding space so that we can add each dimension to x's dimension while adding residual connections (to prevent adding RGB to HSV)
        self.dropout = nn.Dropout(dropout) #think of dropout as a regularization/ optimization technique (kind of like random forest)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    

Self-attention decides how tokens communicate, but the feed-forward network is what actually computes meaningful representations from that information.

In other words: Self-attention gathers information from other tokens, while the feed-forward network turns that information into useful features (this mainly happens because of the presence of non linearity in ffn). After FFN the token would have interpreted the context it received.

In [17]:
class FeedForward(nn.Module):
    #a linear layer followed by a non-linearity

    def __init__(self, n_embd):
        super().__init__() #nn.Module's __init__
        self.net = nn.Sequential(nn.Linear(n_embd, 4*n_embd), nn.ReLU(), nn.Linear(4*n_embd, n_embd),nn.Dropout(dropout),) #think of dropout as a regularization/ optimization technique (kind of like random forest)
        #the last linear layer is what allows the FFN to recombine the ReLU-activated features into a new learned representation that can be added back via the residual connection

    def forward(self, x):
        return self.net(x)

We need to add residual connections because it ensures that gradients cannot vanish completely and information always flows backward.

Early in training, attention weights are nar-random and wihtout residuals, representations can collapse. Rediduals ensure that each layer can only improve or slightly adjust the representation.

Residuals therefore help with stable optimization.

In [18]:
class Block(nn.Module):
    #Transformer block: communication followed by computation

    def __init__(self, n_embd, n_head):
        super().__init__() #nn.Module's __init__
        head_size = n_embd//n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd) #normalises overall the dimensions for 1 token
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) #the x + is to add the residual
        x = x + self.ffwd(self.ln1(x)) #same as above
        return x

In [19]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__() #nn.Module's __init__
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # self.sa_heads = MultiHeadAttention(4,n_embd//4) #i.e., 4 heads of 8-dimensional self-attention
        # self.ffwd = FeedForward(n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) #final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)


    def forward(self, idx, targets=None):
        
        B,T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B,T,C) (number of batches or sequences in parallel, number of tokens being processed, embedding dimension)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #(T,C)
        x = tok_emb + pos_emb #(B,T,C)
        # x = self.sa_heads(x) #apply one head of self-attention (B,T,C)
        # x = self.ffwd(x) #(B,T,C)
        x = self.blocks(x) #(B,T,C)
        x = self.ln_f(x) #(B,T,C)
        logits = self.lm_head(x) #(B,T,C) here C is vocab size. C was 32 earlier now it is 65 as we want logits here

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond) #equivalent to self.forward(idx, targets=None) so (B,T,C)
            logits = logits[:, -1, :] #select last timestamp so it is (B,C), Given the seq so far, how likely is each possible next token
            probs = F.softmax(logits, dim=-1) #(B,C)
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            idx = torch.cat((idx, idx_next), dim=1) #(B,T+1)
        
        return idx


xb, yb = get_batch('train')
model = BigramLanguageModel()
m = model.to(device)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)

torch.Size([16384, 65])
tensor(4.3375, device='cuda:0', grad_fn=<NllLossBackward0>)


In [20]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
     #every now and then evaluate the loss on train and val sets
     if iter % eval_interval == 0:
          losses = estimate_loss()
          print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
     xb, yb = get_batch('train')
     logits, loss = model(xb, yb)
     optimizer.zero_grad(set_to_none = True)
     loss.backward()
     optimizer.step()

context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist())) #[0] becasue we are taking the first element from generate's batch dim (albeit the batch dimension is only 1 rn)

Step 0: train loss 4.3358, val loss 4.3323
Step 500: train loss 1.8958, val loss 2.0136
Step 1000: train loss 1.5430, val loss 1.7318
Step 1500: train loss 1.3993, val loss 1.6093
Step 2000: train loss 1.3112, val loss 1.5537
Step 2500: train loss 1.2550, val loss 1.5269
Step 3000: train loss 1.2050, val loss 1.5071
Step 3500: train loss 1.1623, val loss 1.4891
Step 4000: train loss 1.1263, val loss 1.4872
Step 4500: train loss 1.0900, val loss 1.4830

Brings in this pectition is he sab now
The ground: thou, sit is thou comest any brade,
Yet thou art Ned'st, rootmage.

GREMIO:
But when thou promisest what we dream Was.
And is some fairest petition.
And have thoughts, with the stewder'd wild speak:
That 'twento may speak thee must weach to pale.
But what they dead, that thou toak again
In our corse will not will again; but 'twere
This many consense on your ghost love,
That to be his higherTy, tragger, and yonder Wootman,
His tongue whose cousin


In [22]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=10000)[0].tolist())) #[0] becasue we are taking the first element from generate's batch dim (albeit the batch dimension is only 1 rn)


With truth his trabunes; like a wight posservative
For a madam in a bred, osy steed where my flies,
The torch from my son, our sweetless hand!
O beat o'er my life by thine, wife, Wail I kill himself,
So strok thy lips and course; he, havin
Of they'll present thee most pay on thee.

ESCALUS:
Here in thy nature: 'tis a former sword.

KING RICHARD II:
Warriors, what plainly I now, wife.
Which is the soldier, to close the lire:
He is the fear'st o'erlop'd it as boldly.

KING RICHARD II:
I, in what cousin's head!

SICINGENNEN:
Then give some franks, so, then quickly meet,
For let them  be a most thing your wealt
Infery the subjects of lalls. Perlaisned ANNE:
Where and I reposed for ages, yea, who like himself
From elder the mirthstering of my costage
Than Purgs 'shall young black you find yourself
Is pretty this flishmes no room this rest.
For many business that wrath waste your foe,--
For your arms, and know yours hands, not easy!
Commend me thou I accept of my stars,
Concerns I am alread

In [21]:
# for steps in range(10000):
#     xb, yb = get_batch('train')
#     logits, loss = m(xb,yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step() #upgrade gradient
# print(loss.item())